In [4]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [10]:
data = pd.read_csv('../DataSets/NASA_access_log_Jul95', sep='\n', header = None)

In [16]:
print data[0]

0          199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] ...
1          unicomp6.unicomp.net - - [01/Jul/1995:00:00:06...
2          199.120.110.21 - - [01/Jul/1995:00:00:09 -0400...
3          burger.letters.com - - [01/Jul/1995:00:00:11 -...
4          199.120.110.21 - - [01/Jul/1995:00:00:11 -0400...
5          burger.letters.com - - [01/Jul/1995:00:00:12 -...
6          burger.letters.com - - [01/Jul/1995:00:00:12 -...
7          205.212.115.106 - - [01/Jul/1995:00:00:12 -040...
8          d104.aa.net - - [01/Jul/1995:00:00:13 -0400] "...
9          129.94.144.152 - - [01/Jul/1995:00:00:13 -0400...
10         unicomp6.unicomp.net - - [01/Jul/1995:00:00:14...
11         unicomp6.unicomp.net - - [01/Jul/1995:00:00:14...
12         unicomp6.unicomp.net - - [01/Jul/1995:00:00:14...
13         d104.aa.net - - [01/Jul/1995:00:00:15 -0400] "...
14         d104.aa.net - - [01/Jul/1995:00:00:15 -0400] "...
15         d104.aa.net - - [01/Jul/1995:00:00:15 -0400] "...
16         129.94.144.15